In [1]:
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from skbio.stats.distance import permanova
from skbio import DistanceMatrix

In [4]:
def new_species_detection(test_name, group_root_path, known_species_path,
                          test_species_path, save_path):
    # get a list of group files from group_root_path
    group_list = [file for file in os.listdir(group_root_path)]
    # combine test_name with each group in group_list to create a list of tuples
    result = [(test_name, s) for s in group_list]
    # create an empty dictionary to store p-values for each test
    p_values = {}
    # assume all p-values are less than 0.05 initially
    all_values_small = True

    for a, b in result:
        # read in the test species and known species data as pandas dataframes
        x_1 = pd.read_csv(os.path.join(test_species_path, f"{a}.csv"),
                          index_col=0)
        x_2 = pd.read_csv(os.path.join(known_species_path, f"{b}.csv"),
                          index_col=0)
        # combine the dataframes horizontally
        x = pd.concat([x_1, x_2], axis=1)
        # save the combined dataframes as a new CSV file
        x.to_csv(os.path.join(save_path, f"{a}_{b}_concat.csv"))
        # calculate the pairwise distances between columns (species) of the combined dataframe
        dis = pd.DataFrame(distance_matrix(x.values.T, x.values.T),
                           index=x.columns,
                           columns=x.columns)
        # save the distance matrix as a new CSV file
        dis.to_csv(os.path.join(save_path, f"{a}_{b}_dis.csv"))
        # create a grouping vector for the permanova test (5 samples from groupx and 5 samples from groupy)
        grouping = ["groupx"] * 5 + ["groupy"] * 5
        # create a distance matrix object from the pairwise distances
        Distance = DistanceMatrix(dis, dis.index)
        # run a permanova test using the distance matrix and grouping vector
        adonis = permanova(Distance, grouping, permutations=1000)
        # store the p-value for this test in the dictionary
        p_values[(a, b)] = adonis["p-value"]
        # check if any p-value is greater than or equal to 0.05
        if adonis["p-value"] >= 0.05:
            # if any p-value is greater than or equal to 0.05, set all_values_small to False
            all_values_small = False

    if all_values_small:
        # if all p-values are less than 0.05, output "new species"
        print("new species")
    else:
        # if any p-value is greater than or equal to 0.05, output the corresponding test names and p-values
        for key, value in p_values.items():
            if value >= 0.05:
                print(
                    f"Test species '{key[0]}' and known species '{key[1]}' have a p-value of {value}"
                )

    # return the dictionary of p-values for all tests
    return p_values

In [14]:
test_name = 'Me.ns'
group_root_path = "xxxxxxx"
known_species_path = "xxxxxxxxxx"
test_species_path = "xxxxxxxxx"
save_path = "xxxxxxx"

p_values = new_species_detection(test_name, group_root_path,
                                 known_species_path, test_species_path,
                                 save_path)

new species


In [15]:
p_values

{('Me.ns', 'Me.di'): 0.006993006993006993,
 ('Me.ns', 'Me.ha'): 0.04995004995004995,
 ('Me.ns', 'Me.pr'): 0.006993006993006993}

In [5]:
test_name = 'Me.pr-test'
group_root_path = "xxxxxxx"
known_species_path = "xxxxxxx"
test_species_path = "xxxxxxx"
save_path = "xxxxx"

p_values = new_species_detection(test_name, group_root_path,
                                 known_species_path, test_species_path,
                                 save_path)

Test species 'Me.pr-test' and known species 'Me.pr' have a p-value of 0.5024975024975025


In [6]:
p_values

{('Me.pr-test', 'Me.di'): 0.007992007992007992,
 ('Me.pr-test', 'Me.ha'): 0.005994005994005994,
 ('Me.pr-test', 'Me.pr'): 0.5024975024975025}